# **BeDOZa Protocol With MACs**




In [1036]:
import numpy as np

In [1037]:
p = 41

The definition of the function:

In [1038]:
n = 2
list_of_mults = [(False, 0, 2), (False, 1, 3)]
greater_than = 4

Global auxiliary variables:

In [1039]:
number_of_curr_MULT = 0
d_array = []
e_array = []

## MAC's Class:

In [1040]:
class MAC:
  def __init__(self, x, key, tag):
    self.value = x
    self.key = key
    self.tag = tag

## Node of Circuit's Class
The Node class represents a node in a circuit, which can have two input parents and an operator (op) that defines the operation to be performed on the parents' values. A Circuit is then made up of a collection of Node instances that are interconnected to form a larger computational graph.

In [1041]:
class Node:
  def __init__(self, op, constant, parent1, parent2):
    self.value = None
    self.constant = constant        # contains "False" if the operator is applied to x,y and "True" if the operator is applied to x,c
    self.op = op                    # contains "+" or "*"
    self.parent1 = parent1
    self.parent2 = parent2
    self.sons = []
    self.output = False             # contains "True" if this is leaf in the circuit and "False" otherwise
    self.tag = -1
    self.key = -1

## Circuit's Class:

In [1042]:
class Circuit:
  def __init__(self, n, input, MULTS, number, p):
    self.number_of_mult_gates = 0
    self.n = n
    self.input = input
    self.MULTS = MULTS
    self.p = p
    self.number = number
    self.results_of_greater_than = []

  def Mul(self, parent1, parent2, constant):
    self.number_of_mult_gates += 1
    node_mul = Node("*", constant, parent1, parent2)
    parent1.sons.append(node_mul)
    if constant == False:
      parent2.sons.append(node_mul)
    return node_mul


  def Add(self, parent1, parent2, constant):
    node_add = Node("+", constant, parent1, parent2)
    parent1.sons.append(node_add)
    if constant == False:
      parent2.sons.append(node_add)
    return node_add

  def while_of_visited(self, visited, op, const_flag):
    new_visited = []
    while len(visited) > 1:
      for i in range(0, len(visited), 2):
        if (i + 1) >= len(visited):
          new_visited.append(visited[i])
        else:
          new_visited.append(op(visited[i], visited[i + 1], const_flag))
      visited = new_visited.copy()
      new_visited = []
    return visited[0]

  def GreaterThan(self, parent):
    results_of_power = []

    for i in range(self.number):
      node = self.Add(parent1=parent, parent2=np.negative(i), constant=True)
      results_of_power.append(node)
      for j in range(self.p - 2):
        node_son = self.Mul(parent1=results_of_power[i], parent2=node, constant=False)
        results_of_power[i] = node_son

    result = self.while_of_visited(results_of_power, self.Mul, False)
    return result

  def Not(self, parent):
    node = self.Add(parent, -1, True)
    node_not = self.Mul(node, -1, True)
    return node_not

  def sum(self, parent1, parent2, constant):
    sum = self.Add(parent1, parent2, False)
    self.results_of_greater_than.append(self.GreaterThan(sum))
    return sum

  def GenerateCircuit(self):
    nodes = np.zeros((2 * self.n), dtype=object)
    visited = []
    new_visited = []
    results_of_mul = []
    results_of_greater_than = []
    results_of_not = []
    for i in range(len(self.input)):
        nodes[i] = Node(None, False, None, None)
        nodes[i].value = self.input[i]
    for mult in self.MULTS:
        constant = mult[0]
        parent1 = nodes[mult[1]]
        if constant == False:
            parent2 = nodes[mult[2]]
        else:
            parent2 = mult[2]
        mul = self.Mul(parent1, parent2, constant)
        results_of_mul.append(mul)
    for mul in results_of_mul:
        results_of_greater_than.append(self.GreaterThan(mul))

    self.results_of_greater_than = []
    result = self.while_of_visited(results_of_mul, self.sum, False)
    results_of_greater_than = self.results_of_greater_than
    results_of_greater_than.append(self.GreaterThan(result))

    for node in results_of_greater_than:
      results_of_not.append(self.Not(node))

    result = self.while_of_visited(results_of_not, self.Mul, False)
    result = self.Not(result)
    result.output = True
    return nodes, self.number_of_mult_gates

## Dealer's Class:

In [1043]:
class Dealer:
    """
    Attributes:
      - param circuitA: an object of Circuit that represents an arithmetic circuit of Alice.
      - param circuitB: an object of Circuit that represents an arithmetic circuit of Bob.
      - param number_of_mult_gates: number of mult gates in the circuit.
      - param u: an array of random samples from {0,p-1}.
      - param v: an array of random samples from {0,p-1}.
      - param w: the result of u * v.
      - param u_A: an array of MACs from {0,p-1} such that u = (u_A.value + u_B.value) % p.
      - param v_A: an array of MACs from {0,p-1} such that u = (v_A.value + v_B.value) % p.
      - param w_A: an array of MACs from {0,p-1} such that u = (w_A.value + w_B.value) % p.
      - param u_B: an array of MACs from {0,p-1} such that u = (u_A.value + u_B.value) % p.
      - param v_B: an array of MACs from {0,p-1} such that u = (v_A.value + v_B.value) % p.
      - param w_B: an array of MACs from {0,p-1} such that u = (w_A.value + w_B.value) % p.
      - param r: an array of random samples from {0,p-1}.
      - param r_A: an array of MACs from {0,p-1} such that r = (r_A.value + r_B.value) % p.
      - param r_B: an array of MACs from {0,p-1} such that r = (r_A.value + r_B.value) % p.
    """

    def __init__(self):
      input = np.zeros((2*n), dtype=int)
      circuit = Circuit(n, input, list_of_mults, greater_than, p)
      self.circuitA, self.number_of_mult_gates = circuit.GenerateCircuit()
      self.circuitB, self.number_of_mult_gates = circuit.GenerateCircuit()

      alphaA = np.random.randint(0, p)
      alphaB = np.random.randint(0, p)

      self.u = [np.random.randint(0, p) for i in range(self.number_of_mult_gates)]
      self.v = [np.random.randint(0, p) for i in range(self.number_of_mult_gates)]
      self.w = np.multiply(self.u, self.v)

      self.u_A = [np.random.randint(0, p) for i in range(self.number_of_mult_gates)]
      self.v_A = [np.random.randint(0, p) for i in range(self.number_of_mult_gates)]
      self.w_A = [np.random.randint(0, p) for i in range(self.number_of_mult_gates)]

      self.u_B = np.subtract(self.u, self.u_A) % p
      self.v_B = np.subtract(self.v, self.v_A) % p
      self.w_B = np.subtract(self.w, self.w_A) % p

      self.u_A, self.u_B = self.CreateMACs(inputA=self.u_A, inputB=self.u_B, alphaA=alphaA, alphaB=alphaB)
      self.v_A, self.v_B = self.CreateMACs(inputA=self.v_A, inputB=self.v_B, alphaA=alphaA, alphaB=alphaB)
      self.w_A, self.w_B = self.CreateMACs(inputA=self.w_A, inputB=self.w_B, alphaA=alphaA, alphaB=alphaB)

      self.r = [np.random.randint(0, p) for i in range(2 * n)]
      self.r_A = [np.random.randint(0, p) for i in range(2 * n)]
      self.r_B = np.subtract(self.r, self.r_A) % p

      self.r_A, self.r_B = self.CreateMACs(inputA=self.r_A, inputB=self.r_B, alphaA=alphaA, alphaB=alphaB)

    def RandA(self):
      """
      Returns:
        - circuitA: an object of Circuit that represents an arithmetic circuit of Alice.
        - u_A: an array of MACs from {0,p-1} such that u = (u_A.value + u_B.value) % p.
        - v_A: an array of MACs from {0,p-1} such that v = (v_A.value + v_B.value) % p.
        - w_A: an array of MACs from {0,p-1} such that w = (w_A.value + w_B.value) % p.
      """
      return self.circuitA, self.u_A, self.v_A, self.w_A, self.r_A, self.number_of_mult_gates

    def RandB(self):
      """
      Returns:
        - circuitB: an object of Circuit that represents an arithmetic circuit of Bob.
        - u_B: an array of MACs from {0,p-1} such that u = (u_A + u_B) % p.
        - v_B: an array of MACs from {0,p-1} such that v = (v_A + v_B) % p.
        - w_B: an array of MACs from {0,p-1} such that w = (w_A + w_B) % p.
      """
      return self.circuitB, self.u_B, self.v_B, self.w_B, self.r_B, self.number_of_mult_gates

    def CreateMACs(self, inputA, inputB, alphaA, alphaB):
      """
      Attributes:
        - param inputA: an array of random samples from {0,p-1} of Alice.
        - param inputB: an array of random samples from {0,p-1} of Bob.
        - param alphaA: Alice's alpha from {0,p-1}.
        - param alphaB: Bob's alpha from {0,p-1}.
      Returns:
        - circuitB: an object of Circuit that represents an arithmetic circuit of Bob.
        - u_B: an array of MACs from {0,p-1} such that u = (u_A + u_B) % p.
        - v_B: an array of MACs from {0,p-1} such that v = (v_A + v_B) % p.
        - w_B: an array of MACs from {0,p-1} such that w = (w_A + w_B) % p.
      """
      macs_arrayA = np.zeros(len(inputA), dtype=object)
      macs_arrayB = np.zeros(len(inputB), dtype=object)
      for i in range(len(inputA)):
        x_A = inputA[i]
        x_B = inputB[i]
        beta_A = np.random.randint(0, p)
        beta_B = np.random.randint(0, p)
        tag_A = (alphaB * x_A + beta_B) % p
        tag_B = (alphaA * x_B + beta_A) % p
        macs_arrayA[i] = MAC(x_A, (alphaA, beta_A), tag_A)
        macs_arrayB[i] = MAC(x_B, (alphaB, beta_B), tag_B)
      return macs_arrayA, macs_arrayB

## Alice's Class:

In [1044]:
class Alice:
    """
    Attributes:
      - param circuit: an object of Circuit that represents an arithmetic circuit.
      - param number_of_mult_gates: number of mult gates in the circuit
      - param x: Alice's input.
      - param u_A: an array of MACs from {0,p-1}.
      - param v_A: an array of MACs from {0,p-1}.
      - param w_A: an array of MACs from {0,p-1}.
      - param r_A: an array of MACs from {0,p-1}.
      - param mults: a temporary array that holds the indexes of mult gates in a specific layer.
      - param z_A: an array that holds the leaf node in Alice's circuit.
      - param z_B: an array that Bob sends to Alice and holds the leaf node in Bob's circuit.
      - param visited: a list that contains all the nodes in the next layer.
      - param occurred_nodes: a list that contains all the nodes we have already calculated their value.
    """

    def __init__(self, circuit, x, u_A, v_A, w_A, r_A, number_of_mult_gates):
      self.x = x
      self.circuit = circuit
      self.number_of_mult_gates = number_of_mult_gates

      self.u_A = u_A
      self.v_A = v_A
      self.w_A = w_A

      self.r_A = r_A

      self.mults = []
      self.visited = []
      self.occurred_nodes = []

      self.z_A = None
      self.z_B = None

    def OpenTo(self, node_x_A, node_x_B):
      """
      Attributes:
        - param node_x_A: a node of x_A.
        - param node_x_B: a node of x_B.
      Returns:
        - x: returs OpenTo(x_A, x_B) if Verify(x_B, k_A, t_B) = ACCEPT and False otherwise.
      """
      if type(node_x_A) is np.ndarray and type(node_x_B) is np.ndarray:
        x = np.zeros((len(node_x_A)), dtype=int)
        for i in range(len(node_x_A)):
          x[i] = np.add(node_x_A[i].value, node_x_B[i].value) % p
          if not self.Verify(node_x_B[i].value, node_x_A[i].key, node_x_B[i].tag):
            return False
      else:
        x = np.add(node_x_A.value, node_x_B.value) % p
        if not self.Verify(node_x_B.value, node_x_A.key, node_x_B.tag):
          return False
      return x

    def ReceiveR_BAndSendD(self, r_B):
      """
      Attributes:
        - param r_B: an array of MACs from {0,p-1}.
      Returns:
        - d: an array such that d[i] = x[i] - r[i].
      """
      r = self.OpenTo(self.r_A[:n], r_B)

      d = np.zeros((n), dtype=int)
      for i in range(0, n):
        d[i] = self.x[i] - r[i]
        self.circuit[i].value, self.circuit[i].key, self.circuit[i].tag = self.ADD_with_const(self.r_A[i], d[i])
      return d

    def SendR_A(self):
      """
      Returns:
        - r_A: an array of MACs from {0,p-1}.
      """
      return self.r_A[n:]

    def ReceiveD(self, d):
      """
      Attributes:
        - param d: such that d[i] = y[i] - r[i].
      """
      for i in range(n, 2*n):
        self.circuit[i].value, self.circuit[i].key, self.circuit[i].tag = self.ADD_with_const(self.r_A[i], d[i - n])

      for i in range(len(self.circuit)):
        for son in self.circuit[i].sons:
          self.visited.append(son)
        self.occurred_nodes.append(self.circuit[i])
      return

    def ADD(self, node_x, node_y):
      """
      Attributes:
        - param node_x: a node of x.
        - param node_y: a node of y.
      Returns:
        - z_A: x_A + y_A.
        - k_Az: (alpha_x, beta_x + beta_y).
        - t_Az: t_x + t_y.
      """
      key_x = node_x.key
      key_y = node_y.key

      z_A = (node_x.value + node_y.value) % p
      k_Az = (key_x[0] % p, (key_x[1] + key_y[1]) % p)
      t_Az = (node_x.tag + node_y.tag) % p
      return z_A, k_Az, t_Az

    def ADD_with_const(self, node_x, c):
      """
      Attributes:
        - param node_x: a node of x.
        - param c: a value of c.
      Returns:
        - z_A: x_A + c.
        - k_Az: (alpha_x, beta_x).
        - t_Az: t_x.
      """
      key_x = node_x.key

      z_A = (node_x.value + c) % p
      k_Az = (key_x[0] % p, key_x[1] % p)
      t_Az = node_x.tag % p
      return z_A, k_Az, t_Az

    def MULT(self, node_x, node_y, u_A, v_A, w_A, d, e):
      """
      Attributes:
        - param node_x: a node of x.
        - param node_y: a node of y.
        - u_A: a MAC.
        - v_A: a MAC.
        - w_A: a MAC.
        - d: OpenTo(d_A, d_B).
        - e: OpenTo(e_A, e_B).
      Returns:
        - z_A: x_A * y_A.
        - k_Az: (alpha_x, beta_new).
        - t_Az: t_new.
      """
      key_x = node_x.key
      key_y = node_y.key

      z_mul1, k_mul1, t_mul1 = self.MULT_with_const(u_A, e)
      z_mul2, k_mul2, t_mul2 = self.MULT_with_const(v_A, d)

      mac1 = MAC(z_mul1, k_mul1, t_mul1)
      mac2 = MAC(z_mul2, k_mul2, t_mul2)

      z_add1, k_add1, t_add1 = self.ADD(mac1, mac2)
      mac3 = MAC(z_add1, k_add1, t_add1)

      z_add2, k_add2, t_add2 = self.ADD(w_A, mac3)
      mac4 = MAC(z_add2, k_add2, t_add2)

      z_A, k_Az, t_Az = self.ADD_with_const(mac4, e*d)
      return z_A, k_Az, t_Az

    def MULT_with_const(self, node_x, c):
      """
      Attributes:
        - param node_x: a node of x.
        - param c: a value of c.
      Returns:
        - z_A: x_A * c.
        - k_Az: (alpha_x, beta_x * c).
        - t_Az: t_x * c.
      """
      key_x = node_x.key

      z_A = (node_x.value * c) % p
      k_Az = (key_x[0] % p, (key_x[1] * c) % p)
      t_Az = (node_x.tag * c) % p
      return z_A, k_Az, t_Az

    def Receive(self, z_B):
        """
        Attributes:
          param z_B: an array of the leaf in Bob's circuit.
        """
        global d_array, e_array

        self.z_B = z_B

        for mult_node in self.mults:
            node_x_A = mult_node.parent1
            node_y_A = mult_node.parent2
            number_of_current_MULT = mult_node.value
            mult_node.value, mult_node.key, mult_node.tag = self.MULT(node_x_A, node_y_A, self.u_A[number_of_current_MULT], self.v_A[number_of_current_MULT], self.w_A[number_of_current_MULT], d_array[0], e_array[0])
            del d_array[0]
            del e_array[0]
        d_array = []
        e_array = []
        self.mults.clear()

    def Send(self):
      """
      Returns:
        - MULTS: an array that holds the indexes of mult gates in a specific layer.
      """
      global number_of_curr_MULT

      MULTS = []
      new_visited = []
      
      if len(self.visited) == 1:
        self.z_A = self.visited[0]

      for node in self.visited:
        node_x_A = node.parent1
        if node.constant == False:
          node_y_A = node.parent2
        else:
          c = node.parent2

        if node.value is None:
          if node.op == "+" and node.constant:
            node.value, node.key, node.tag = self.ADD_with_const(node_x_A, c)

          if node.op == "+" and not node.constant:
            node.value, node.key, node.tag = self.ADD(node_x_A, node_y_A)

          if node.op == "*" and node.constant:
            node.value, node.key, node.tag = self.MULT_with_const(node_x_A, c)

          if node.op == "*" and not node.constant:
            self.mults.append(node)
            d_A = np.subtract(node_x_A.value, self.u_A[number_of_curr_MULT].value) % p
            e_A = np.subtract(node_y_A.value, self.v_A[number_of_curr_MULT].value) % p
            node.value = number_of_curr_MULT
            MULTS.append([d_A, e_A, number_of_curr_MULT])
            number_of_curr_MULT += 1
              
          self.occurred_nodes.append(node)
          if len(node.sons) != 0:
            for son in node.sons:
              if son.parent1 in self.occurred_nodes and (son.constant or son.parent2 in self.occurred_nodes):
                new_visited.append(son)

      self.visited = new_visited.copy()
      new_visited.clear()
      return MULTS

    def hasOutput(self):
      """
      Returns:
        - True/False: a Boolean value indicating whether Alice has input.
      """
      if self.z_B is not None:
        return True
      return False

    def Output(self):
      """
      Returns:
        - z: Alice's output.
      """
      z = self.OpenTo(self.z_A, self.z_B[0])
      return z

    def Verify(self, x, k, t):
      """
      Attributes:
        - param x: a value of x.
        - param k: a tuple of key.
        - param t: a tag.
      Returns:
        - True or False: True if Ver(x,k,t) = ACCEPT and False if Ver(x,k,t) = ABORT.
      """
      if (k[0] * x + k[1]) % p == t:
        return True
      else:
        return False

## Bob's Class:

In [1045]:
class Bob:
    """
    Attributes:
      - param circuit: an object of Circuit that represents an arithmetic circuit.
      - param number_of_mult_gates: number of mult gates in the circuit
      - param y: Bobs's input.
      - param u_B: an array of MACs from {0,p-1}.
      - param v_B: an array of MACs from {0,p-1}.
      - param w_B: an array of MACs from {0,p-1}.
      - param r_B: an array of MACs from {0,p-1}.
      - param z_B: an array that Bob sends to Alice and holds the leaf node in Bob's circuit.
      - param visited: a list that contains all the nodes in the next layer.
      - param occurred_nodes: a list that contains all the nodes we have already calculated their value.
    """

    def __init__(self, circuit, y, u_B, v_B, w_B, r_B, number_of_mult_gates):
      self.y = y
      self.circuit = circuit
      self.number_of_mult_gates = number_of_mult_gates

      self.u_B = u_B
      self.v_B = v_B
      self.w_B = w_B

      self.r_B = r_B
      
      self.visited = []
      self.occurred_nodes = []

      self.z_B = None

    def OpenTo(self, node_x_A, node_x_B):
      """
      Attributes:
        - param node_x_A: a node of x_A.
        - param node_x_B: a node of x_B.
      Returns:
        - x: returs OpenTo(x_A, x_B) if Verify(x_A, k_B, t_A) = ACCEPT and False otherwise.
      """
      if type(node_x_A) is np.ndarray and type(node_x_B) is np.ndarray:
        x = np.zeros((len(node_x_A)), dtype=int)
        for i in range(len(node_x_A)):
          x[i] = np.add(node_x_A[i].value, node_x_B[i].value) % p
          if not self.Verify(node_x_A[i].value, node_x_B[i].key, node_x_A[i].tag):
            return False
      else:
        x = np.add(node_x_A.value, node_x_B.value) % p
        if not self.Verify(node_x_A.value, node_x_B.key, node_x_A.tag):
          return False
      return x

    def SendR_B(self):
      """
      Returns:
        - r_B: an array of MACs from {0,p-1}.
      """
      return self.r_B[:n]

    def ReceiveD(self, d):
      """
      Attributes:
        - param d: such that d[i] = x[i] - r[i].
      """
      for i in range(0,n):
        self.circuit[i].value, self.circuit[i].key, self.circuit[i].tag = self.ADD_with_const(self.r_B[i], d[i])

    def ReceiveR_AAndSendD(self, r_A):
      """
      Attributes:
        - param r_A: an array of MACs from {0,p-1}.
      Returns:
        - d: an array such that d[i] = y[i] - r[i].
      """
      r = self.OpenTo(r_A, self.r_B[n:])

      d = np.zeros((n), dtype=int)
      for i in range(0, n):
        d[i] = self.y[i] - r[i]
        self.circuit[i + n].value, self.circuit[i + n].key, self.circuit[i + n].tag = self.ADD_with_const(self.r_B[i + n], d[i])

      for i in range(len(self.circuit)):
        for son in self.circuit[i].sons:
          self.visited.append(son)
        self.occurred_nodes.append(self.circuit[i])
      return d

    def ADD(self, node_x, node_y):
      """
      Attributes:
        - param node_x: a node of x.
        - param node_y: a node of y.
      Returns:
        - z_B: x_B + y_B.
        - k_Bz: (alpha_x, beta_x + beta_y).
        - t_Bz: t_x + t_y.
      """
      key_x = node_x.key
      key_y = node_y.key

      z_B = (node_x.value + node_y.value) % p
      k_Bz = (key_x[0] % p, (key_x[1] + key_y[1]) % p)
      t_Bz = (node_x.tag + node_y.tag) % p
      return z_B, k_Bz, t_Bz

    def ADD_with_const(self, node_x, c):
      """
      Attributes:
        - param node_x: a node of x.
        - param c: a value of c.
      Returns:
        - z_B: x_B + c.
        - k_Bz: (alpha_x, beta_x - c * alpha_x).
        - t_Bz: t_x.
      """
      key_x = node_x.key

      z_B = (node_x.value) % p
      k_Bz = (key_x[0] % p, (key_x[1] - c * key_x[0]) % p)
      t_Bz = node_x.tag % p
      return z_B, k_Bz, t_Bz

    def MULT(self, node_x, node_y, u_B, v_B, w_B, d, e):
      """
      Attributes:
        - param node_x: a node of x.
        - param node_y: a node of y.
        - u_B: a MAC.
        - v_B: a MAC.
        - w_B: a MAC.
        - d: OpenTo(d_A, d_B).
        - e: OpenTo(e_A, e_B).
      Returns:
        - z_B: x_B * y_B.
        - k_Bz: (alpha_x, beta_new).
        - t_Bz: t_new.
      """
      key_x = node_x.key
      key_y = node_y.key

      z_mul1, k_mul1, t_mul1 = self.MULT_with_const(u_B, e)
      z_mul2, k_mul2, t_mul2 = self.MULT_with_const(v_B, d)

      mac1 = MAC(z_mul1, k_mul1, t_mul1)
      mac2 = MAC(z_mul2, k_mul2, t_mul2)

      z_add1, k_add1, t_add1 = self.ADD(mac1, mac2)
      mac3 = MAC(z_add1, k_add1, t_add1)

      z_add2, k_add2, t_add2 = self.ADD(w_B, mac3)
      mac4 = MAC(z_add2, k_add2, t_add2)

      z_B, k_Bz, t_Bz = self.ADD_with_const(mac4, e*d)
      return z_B, k_Bz, t_Bz

    def MULT_with_const(self, node_x, c):
      """
      Attributes:
        - param node_x: a node of x.
        - param c: a value of c.
      Returns:
        - z_B: x_B * c.
        - k_Bz: (alpha_x, beta_x * c).
        - t_Bz: t_x * c.
      """
      key_x = node_x.key

      z_B = (node_x.value * c) % p
      k_Bz = (key_x[0] % p, (key_x[1] * c) % p)
      t_Bz = (node_x.tag * c) % p
      return z_B, k_Bz, t_Bz

    def Receive(self, MULTS):
      """
      Attributes:
        - param MULTS: an array that holds the indexes of mult gates in a specific layer.
      """
      global d_array, e_array

      new_visited = []

      for node in self.visited:
        node_x_B = node.parent1
        if node.constant == False:
          node_y_B = node.parent2
        else:
          c = node.parent2

        if node.value is None:
          if node.op == "+" and node.constant:
            node.value, node.key, node.tag = self.ADD_with_const(node_x_B, c)

          if node.op == "+" and not node.constant:
            node.value, node.key, node.tag = self.ADD(node_x_B, node_y_B)

          if node.op == "*" and node.constant:
            node.value, node.key, node.tag = self.MULT_with_const(node_x_B, c)
            
          if node.op == "*" and not node.constant:
            number_of_current_MULT = MULTS[0][2]
            d_A = MULTS[0][0]
            d_B = np.subtract(node_x_B.value, self.u_B[number_of_current_MULT].value) % p
            d_array.append(d_A + d_B)

            e_A = MULTS[0][1]
            e_B = np.subtract(node_y_B.value, self.v_B[number_of_current_MULT].value) % p
            e_array.append(e_A + e_B)

            del MULTS[0]

            node.value, node.key, node.tag = self.MULT(node_x_B, node_y_B, self.u_B[number_of_current_MULT], self.v_B[number_of_current_MULT], self.w_B[number_of_current_MULT], d_array[-1], e_array[-1])

          self.occurred_nodes.append(node)
          if len(node.sons) != 0:
            for son in node.sons:
              if son.parent1 in self.occurred_nodes and (son.constant == True or son.parent2 in self.occurred_nodes):
                new_visited.append(son)
            self.z_B = new_visited.copy()

      self.visited = new_visited.copy()
      new_visited.clear()

    def Send(self):
      """
        Returns:
          - z_B: an array of the leaf in Bob's circuit.
      """
      if len(self.visited) > 0:
        return None
      else:
        return self.z_B

    def Verify(self, x, k, t):
      """
      Attributes:
        - param x: a value of x.
        - param k: a tuple of key.
        - param t: a tag.
      Returns:
        - True or False: True if Ver(x,k,t) = ACCEPT and False if Ver(x,k,t) = ABORT.
      """
      if (k[0] * x + k[1]) % p == t:
        return True
      else:
        return False

## The Protocol:

### Generate $\vec{a}=(a_1,a_2) \in {\{0,1,2,3\}^2}$ and $\vec{x}=(x_1,x_2) \in {\{0,1,2,3\}^2}$

In [1046]:
a_vec = [np.random.randint(0,4) for i in range(n)]
x_vec = [np.random.randint(0,4) for i in range(n)]

print("a =", a_vec)
print("x =", x_vec)

a = [3, 0]
x = [2, 1]


### Offline Phase:

In [1047]:
dealer = Dealer()
circuit_A, u_A, v_A, w_A, r_A, mult_gates_A = dealer.RandA()
circuit_B, u_B, v_B, w_B, r_B, mult_gates_B = dealer.RandB()

### Online Phase:

Initialization of Alice and Bob:

In [1048]:
Alice = Alice(circuit=circuit_A, x=x_vec, u_A=u_A, v_A=v_A, w_A=w_A, r_A=r_A, number_of_mult_gates=mult_gates_A)
Bob = Bob(circuit=circuit_B, y=a_vec, u_B=u_B, v_B=v_B, w_B=w_B, r_B=r_B, number_of_mult_gates=mult_gates_B)

Secret sharing:

In [1049]:
d = Alice.ReceiveR_BAndSendD(r_B=Bob.SendR_B())
Bob.ReceiveD(d=d)

d = Bob.ReceiveR_AAndSendD(r_A = Alice.SendR_A())
Alice.ReceiveD(d=d)

In [1050]:
while Alice.hasOutput() == False:
  MULTS = Alice.Send()
  Bob.Receive(MULTS=MULTS)
  z_B = Bob.Send()
  Alice.Receive(z_B=z_B)

Alice outputs:

In [1051]:
z = Alice.Output()
print("z =", z)

z = 1


### In short:

for $a = (a_{1},a_{2}), x = (x_{1},x_{2})$:


In [1052]:
print("a =", a_vec)
print("x =", x_vec)

a = [3, 0]
x = [2, 1]


The output of $f_{a⃗,4}(x_1,x_2)$ is:

In [1053]:
print(z)

1
